In [ ]:
import folium
import json
from kafka import KafkaConsumer
import time
from geopy.geocoders import Nominatim
import tkinter as tk

SERVER = "broker:9092"
TOPIC = "car_test1"

consumer = KafkaConsumer(
    TOPIC,
    bootstrap_servers=[SERVER],
    auto_offset_reset="earliest",
    enable_auto_commit=True,
    group_id="my-group",
    value_deserializer=lambda x: json.loads(x.decode("utf-8")),
    api_version=(3, 7, 0),
)


In [ ]:
# Geocoder
geolocator = Nominatim(user_agent="car_simulation")

def show_popup(car_lat, car_lon, country):
    root = tk.Tk()
    root.title("Car Position Information")

    msg = f"Car Position: {car_lat}, {car_lon}\nCountry: {country}"
    label = tk.Label(root, text=msg, padx=20, pady=20, font=("Helvetica", 12))
    label.pack()

    button = tk.Button(root, text="OK", command=root.destroy, padx=10, pady=5)
    button.pack(pady=10)

    root.mainloop()

try:
    for message in consumer:
        print(message)
        data = message.value

        # Get actual car position from producer (x=lon, y=lat)
        car_pos = data["Car"]
        car_lat = car_pos[1]  # y
        car_lon = car_pos[0]  # x

        # Reverse geocoding: get country from coordinates
        location = geolocator.reverse((car_lat, car_lon), language='en')
        country = "Unknown"
        if location:
            address = location.raw.get('address', {})
            country = address.get('country', "Unknown")

        # Create map centered on actual car location
        mymap = folium.Map(location=[car_lat, car_lon], zoom_start=13)

        folium.Marker(
            location=[car_lat, car_lon],
            popup=f"Car position: {car_lat}, {car_lon}\nCountry: {country}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(mymap)

        # Optional: Save the updated map (uncomment to enable)
        # mymap.save("car_simulation_map.html")

        show_popup(car_lat, car_lon, country)
        time.sleep(1)

except KeyboardInterrupt:
    print("\nConsumer stopped.")
finally:
    consumer.close()
    print("Consumer closed.")


Consumer stopped.
Consumer closed.


In [ ]:
from kafka import KafkaConsumer
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

global_errors = {
    "GPS1": 0.0,
    "GPS2": 0.0,
    "Combined": 0.0
}

distances = {
    "GPS1": [],
    "GPS2": [],
    "Combined": []
}

def combine_positions(gps1, gps2):
    combined_x = (gps1[0] + gps2[0]) / 2
    combined_y = (gps1[1] + gps2[1]) / 2
    return (combined_x, combined_y)

def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def main():
    SERVER = "broker:9092"
    TOPIC = "car_test1"

    consumer = KafkaConsumer(
        TOPIC,
        bootstrap_servers=[SERVER],
        auto_offset_reset="earliest",
        enable_auto_commit=True,
        group_id="my-group",
        value_deserializer=lambda x: json.loads(x.decode("utf-8")),
        api_version=(3, 7, 0),
    )

    fig, ax = plt.subplots()
    x_data, y_data_gps1, y_data_gps2, y_data_combined = [], [], [], []

    def update(frame):
        for message in consumer:
            data = message.value
            car_pos = data["Car"]
            gps1_pos = data["GP1"]
            gps2_pos = data["GP2"]

            distance_gps1_to_car = calculate_distance(car_pos, gps1_pos)
            distance_gps2_to_car = calculate_distance(car_pos, gps2_pos)
            
            combined_pos = combine_positions(gps1_pos, gps2_pos)
            distance_combined_to_car = calculate_distance(car_pos, combined_pos)

            global_errors["GPS1"] += distance_gps1_to_car
            global_errors["GPS2"] += distance_gps2_to_car
            global_errors["Combined"] += distance_combined_to_car
            
            distances["GPS1"].append(global_errors["GPS1"])
            distances["GPS2"].append(global_errors["GPS2"])
            distances["Combined"].append(global_errors["Combined"])
            
            x_data.append(len(x_data))
            y_data_gps1.append(global_errors["GPS1"])
            y_data_gps2.append(global_errors["GPS2"])
            y_data_combined.append(global_errors["Combined"])
            
            ax.clear()
            ax.plot(x_data, y_data_gps1, label='GPS1 Error')
            ax.plot(x_data, y_data_gps2, label='GPS2 Error')
            ax.plot(x_data, y_data_combined, label='Combined Error')
            
            ax.legend()
            ax.set_xlabel('Time')
            ax.set_ylabel('Cumulative Error')
            ax.set_title('Real-time GPS Error Tracking')
            break

    ani = FuncAnimation(fig, update, interval=1000, cache_frame_data=False)
    ani.save('gps_errors.gif', writer='pillow')

    try:
        plt.show()
    except KeyboardInterrupt:
        consumer.close()

if __name__ == "__main__":
    main()


Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x7f55a1562de0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/matplotlib/transforms.py", line 198, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id_self: pop(k))

KeyboardInterrupt: 


In [6]:
!pip install geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [geopy]32m1/2 [geopy]


In [3]:
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [folium]2m1/2 [folium]


In [1]:
!pip install tk